In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
load_dotenv()

True

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [11]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [14]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("cual es el clima en  bogota")
print(search_results)

[{'url': 'https://www.tiempo.com/bogota.htm', 'content': 'El Tiempo en Bogotá - Previsión meteorológica para los próximos 14 días. El pronóstico del tiempo más actualizado en Bogotá: temperatura, lluvia, viento, etc'}, {'url': 'https://weather.com/es-CO/tiempo/horario/l/Bogotá?canonicalCityId=efb63aa836e7633fda3cf21e17582fac5c0911eb484a12ce58b3b0bdaeb398b5', 'content': 'recents\nPronósticos especiales\nPronóstico por hora-Bogotá\nmartes, 30 de enero\n3:00\nMuy nublado\n4:00\nNublado\n5:00\nNublado\n6:00\nMuy nublado\n7:00\nParcialmente nublado\n8:00\nParcialmente nublado\n9:00\nParcialmente nublado\n10:00\nParcialmente nublado\n11:00\nParcialmente nublado\n12:00\nParcialmente nublado\n13:00\nParcialmente nublado\n14:00\nParcialmente nublado\n15:00\nMuy nublado\n16:00\nMuy nublado\n17:00\nMuy nublado\n18:00\nMuy nublado\n19:00\nParcialmente nublado\n20:00\nParcialmente nublado\n21:00\nMuy nublado\n22:00\nParcialmente nublado\n23:00\nParcialmente nublado\nmiércoles, 31 de enero\n0:00\nPa

In [15]:
tools = [search]

In [17]:
response = model.invoke([HumanMessage("hola como estas")])
response

AIMessage(content='¡Hola! Estoy bien, gracias por preguntar. ¿Y tú, cómo estás? \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-993c3e00-9d1f-43fc-8a6a-b4ff80cb8255-0', usage_metadata={'input_tokens': 4, 'output_tokens': 17, 'total_tokens': 21})

In [18]:
model_with_tools = model.bind_tools(tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [19]:
response = model_with_tools.invoke([HumanMessage("hola como estas")])
print(f"Content string: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

Content string: I am an AI and do not have feelings. How can I help you today? 

ToolCalls: []


In [20]:
response = model_with_tools.invoke([HumanMessage("cual es el clima en bogota")])
print(f"Content string: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

Content string: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in bogota'}, 'id': '81f7694d-a36e-4b2b-932c-d393b5355876', 'type': 'tool_call'}]


In [21]:
# construir agente con langgraph
agent_executor = create_react_agent(model=model, tools=tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [22]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Hola como estas")]})

response["messages"]

[HumanMessage(content='Hola como estas', id='67576721-0e6e-48ef-8ebf-cf5c3aa65279'),
 AIMessage(content='¡Hola! Estoy bien, gracias por preguntar. ¿Y tú? 😊 \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-84e6fc1f-7ba2-4c34-9927-540c2954b2f2-0', usage_metadata={'input_tokens': 77, 'output_tokens': 15, 'total_tokens': 92})]

In [23]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="cual es el clima hoy en bogota?")]}
)
response["messages"]

[HumanMessage(content='cual es el clima hoy en bogota?', id='cd5b9de8-edab-46ce-af13-94df3d10e0f4'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in bogota today"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-5d8962c8-3a26-4261-81f3-10bb3affbbb9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in bogota today'}, 'id': 'fd0b307f-d376-4467-9a99-2fa39d8a71e5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 8

# Streaming Messages

In [28]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="cual es el clima hoy en bogota?")]}
):
    print(chunk)
    print("------")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in bogota today"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-943ef7b5-0cb2-4faa-866e-b27a44d83cfb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in bogota today'}, 'id': 'cd8137c3-842e-4b43-bd96-dff689e3b9ab', 'type': 'tool_call'}], usage_metadata={'input_tokens': 83, 'output_tokens': 24, 'total_tokens': 107})]}}
------
{'tools': {'messages':

# Streaming tokens

In [39]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="cual es el clima hoy en bogota?")]}, version="v1"
):
    kind = event["event"]
    name = event["name"]
    if kind == "on_chain_start":
        if ( name == "agent" ):
            print(f"Iniciando agente {name} con la entrada: {event['data'].get('input')}")
    elif kind == "on_chain_end":
        if ( name == "agent" ):
            print()
            print("---")
            print(f"Finalizando agente {name} con la salida: {event['data'].get('output')}")
    elif kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            print(content, end="|")
    elif kind == "on_chain_stream":
        content = event["data"]["chunk"].get("messages")
        if content:
            f"Starting tool: {name} with contet: {content}"
    elif kind == "on_chain_end":
        content = event["data"]["chunk"].content
        if content:
            print(content, end="|")

Iniciando agente agent con la entrada: None

---
Finalizando agente agent con la salida: {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in bogota today"}'}}, response_metadata={'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-6621ae6f-ebe1-415e-9560-8cf12574b6e6', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in bogota today'}, 'id': 'b02a7d50-779b-4bf5-bc1c-c097576ef081', 'type': 'tool_call'}], usage_metadata={'input_tokens': 83, 'output_tokens': 24, 'total_tokens': 107})]}
Iniciando agent

In [34]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="cual es el clima hoy en bogota?")]}, version="v1"
):
    kind = event["event"]
    print(event)

{'event': 'on_chain_start', 'run_id': '0508729f-8441-4d1f-a19c-fd0663bc0109', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='cual es el clima hoy en bogota?')]}}, 'parent_ids': []}
{'event': 'on_chain_start', 'name': '__start__', 'run_id': 'ccdf77f3-827d-47b3-8777-b88030801ae1', 'tags': ['graph:step:0', 'langsmith:hidden'], 'metadata': {'langgraph_step': 0, 'langgraph_node': '__start__', 'langgraph_triggers': ['__start__'], 'langgraph_task_idx': 0, 'thread_ts': '1ef4c92c-8f46-6c8c-bffe-d1c7d67e5cde'}, 'data': {'input': {'messages': [HumanMessage(content='cual es el clima hoy en bogota?')]}}, 'parent_ids': []}
{'event': 'on_chain_end', 'name': '__start__', 'run_id': 'ccdf77f3-827d-47b3-8777-b88030801ae1', 'tags': ['graph:step:0', 'langsmith:hidden'], 'metadata': {'langgraph_step': 0, 'langgraph_node': '__start__', 'langgraph_triggers': ['__start__'], 'langgraph_task_idx': 0, 'thread_ts': '1ef4c92c-8f46-6c8c-bffe-d1c7d67e5cde'}, 'da